# Imports and Loading Data

In [21]:
!pip3 install pytorch-pretrained-bert
!pip3 install simplediff
!pip3 install tensorboardX

     |████████████████████████████████| 317kB 4.6MB/s 


In [22]:
#from pytorch_pretrained_bert.modeling import PreTrainedBertModel, BertModel, BertSelfAttention
import pytorch_pretrained_bert.modeling as modeling
import torch
import torch.nn as nn
import numpy as np
import copy
from collections import defaultdict
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

from tqdm import tqdm
import sys

import pickle
import os
from pytorch_pretrained_bert.modeling import BertForTokenClassification
from torch.nn import CrossEntropyLoss
from tensorboardX import SummaryWriter
import argparse
import sklearn.metrics as metrics

In [47]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_pretrained_bert.modeling import BertModel, BertSelfAttention
from pytorch_pretrained_bert.modeling import BertPreTrainedModel

In [23]:
#from shared.args import ARGS 
#from shared.constants import CUDA 
#import seq2seq.model as seq2seq_model
CUDA = (torch.cuda.device_count() > 0)

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [29]:
## Update the experiments directory
DATA_DIRECTORY = '/content/drive/Shared drives/EC463 464 Senior Design Project/data/'
LEXICON_DIRECTORY = DATA_DIRECTORY+ 'lexicons/'
PRYZANT_DATA = DATA_DIRECTORY + 'pryzant_data/WNC/'
#IMPORTS = 
training_data = PRYZANT_DATA + 'biased.word.train'
testing_data = PRYZANT_DATA + 'biased.word.test'
categories_file = PRYZANT_DATA + 'revision_topics.csv'
pickle_directory = '/content/drive/Shared drives/EC463 464 Senior Design Project/data/pickle_data/'

In [19]:
# Load imports
!cp '/content/drive/Shared drives/EC463 464 Senior Design Project/imports/data.py' .
import data

In [24]:
os.getcwd()

'/content'

In [25]:
print('LOADING DATA...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', os.getcwd() + '/cache')
tok2id = tokenizer.vocab
tok2id['<del>'] = len(tok2id)

LOADING DATA...


100%|██████████| 231508/231508 [00:00<00:00, 1158820.30B/s]


In [30]:
train_dataloader, num_train_examples = data.get_dataloader(
    data_path=training_data,
    tok2id=tok2id,
    batch_size=32,
    pickle_path=pickle_directory + 'train_data.p',
    categories_path=categories_file
  )

eval_dataloader, num_eval_examples = data.get_dataloader(
    data_path=testing_data,
    tok2id=tok2id,
    batch_size=32,
    pickle_path=pickle_directory + 'test_data.p',
    categories_path=categories_file
  )

53803it [00:11, 4818.17it/s]


SKIPPED  1503


1000it [00:01, 970.16it/s]


SKIPPED  32


In [44]:
for step, x in enumerate(train_dataloader):
  #print(x.size())
  print(x)
  print(x[0])
  print(x[0][0])
  
  input()

AttributeError: ignored

# Define Model

In [48]:
class BertForMultitask(BertPreTrainedModel):

    def __init__(self, config, cls_num_labels=2, tok_num_labels=2, tok2id=None):
        super(BertForMultitask, self).__init__(config)
        self.bert = BertModel(config)

        self.cls_dropout = nn.Dropout(config.hidden_dropout_prob)
        self.cls_classifier = nn.Linear(config.hidden_size, cls_num_labels)
        
        self.tok_dropout = nn.Dropout(config.hidden_dropout_prob)
        self.tok_classifier = nn.Linear(config.hidden_size, tok_num_labels)
        
        self.apply(self.init_bert_weights)


    def forward(self, input_ids, token_type_ids=None, attention_mask=None, 
        labels=None, rel_ids=None, pos_ids=None, categories=None, pre_len=None):
        global ARGS
        sequence_output, pooled_output = self.bert(
            input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)

        cls_logits = self.cls_classifier(pooled_output)
        cls_logits = self.cls_dropout(cls_logits)

        # NOTE -- dropout is after proj, which is non-standard
        tok_logits = self.tok_classifier(sequence_output)
        tok_logits = self.tok_dropout(tok_logits)

        return cls_logits, tok_logits

In [53]:
class TaggerFromDebiaser(nn.Module):
    def __init__(self, cls_num_labels=2, tok_num_labels=2, hidden_size=512, tok2id=None):
        super(TaggerFromDebiaser, self).__init__()

        #global ARGS
        #global CUDA

        #if ARGS.pointer_generator:
        #    self.debias_model = seq2seq_model.PointerSeq2Seq(
        #        vocab_size=len(tok2id), hidden_size=hidden_size,
        #        emb_dim=768, dropout=0.2, tok2id=tok2id)
        #else:
        # GET THE SEQ2SEQ MODEL!!!!
        self.debias_model = seq2seq_model.Seq2Seq(
            vocab_size=len(tok2id), hidden_size=hidden_size,
            emb_dim=768, dropout=0.2, tok2id=tok2id)

        #assert ARGS.debias_checkpoint
        #print('LOADING DEBIASER FROM ' + ARGS.debias_checkpoint)
        #self.debias_model.load_state_dict(torch.load(ARGS.debias_checkpoint))
        #print('...DONE')

        self.cls_classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Dropout(0.1),
            nn.ReLU(),
            nn.Linear(hidden_size, cls_num_labels),
            nn.Dropout(0.1))

        self.tok_classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Dropout(0.1),
            nn.ReLU(),
            nn.Linear(hidden_size, tok_num_labels),
            nn.Dropout(0.1))


    def forward(self, input_ids, token_type_ids=None, attention_mask=None,
        labels=None, rel_ids=None, pos_ids=None, categories=None, pre_len=None):

        pre_mask = 1.0-attention_mask

        # src_outputs is [batch_size, sequence_length, hidden_size].
        src_outputs, h_t, _ = self.debias_model.run_encoder(
            input_ids, pre_len, pre_mask)

        cls_logits = self.cls_classifier(h_t)
        tok_logits = self.tok_classifier(src_outputs)

        return cls_logits, tok_logits

In [54]:
model = TaggerFromDebiaser(
        cls_num_labels=43, tok_num_labels=3,
        tok2id=tok2id)

NameError: ignored